In [1]:
import time
from bs4 import BeautifulSoup
import random
import cloudscraper
import requests

def page(urlpage):
    """
    Récupération du HTML d'un site internet en prenant des précautions contre le blocage.
    """
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    ]
    
    headers = {'User-Agent': random.choice(user_agents)}
    
    # Attendre un délai aléatoire pour éviter d'être détecté comme un bot
    time.sleep(random.uniform(3, 7))
    
    scraper = cloudscraper.create_scraper()
    
    try:
        res = scraper.get(urlpage, headers=headers, timeout=10)
        res.raise_for_status()  # Vérifie s'il y a une erreur HTTP (ex: 403, 500, etc.)
    except Exception as e:
        print(f"❌ Erreur lors de la récupération de la page : {e}")
        return None

    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def page_old(urlpage):
    """
    Récupération du HTML d'un site internet via Beautifulsoup
    """
    user_agent = {
        'User-Agent':
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')
        }
    time.sleep(3)
    res = requests.get(urlpage, headers=user_agent)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

In [7]:
!pip install webdriver-manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [11]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options

# Configuration de Firefox en mode headless (sans interface graphique)
options = Options()
options.headless = True  # Mettre False si vous voulez voir le navigateur s'ouvrir

# Initialisation du WebDriver pour Firefox
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)

# URL cible
url = "https://fr.whoscored.com/matches/1891195/live/europe-europa-league-2024-2025-eintracht-frankfurt-ajax"

# Accéder à la page
driver.get(url)

# Attendre le chargement de la page (optionnel, utile si la page charge dynamiquement des éléments)
driver.implicitly_wait(5)

# Récupérer le HTML de la page
html = driver.page_source
# Fermer le navigateur
driver.quit()

soup = BeautifulSoup(html, "html.parser")

In [12]:
soup

<html lang="fr"><head><style> .webpush-window-body { height: 100%; min-height: 100%; padding: 0; margin: 0;}#webpush-window { position: absolute; text-align: center; width: 100%; left: 0; top: 50%; opacity: 0;}#webpush-window-image { height: 128px; margin-bottom: 10px;}#webpush-window-title { font-size: 22px; padding: 0 5px;}#webpush-window-text { color: #333333; margin-top: 15px; padding: 0 5px; line-height: 22px;}#webpush-custom-prompt { position: fixed; z-index: 9999999; width: 320px; max-width: 100%; padding: 10px; top: 0; left: 50%; margin-left: -170px; background-color: rgb(231,232,233); box-shadow: 0px 0px 6px #888888; color: #000000; font-family: 'Trebuchet MS', Helvetica, sans-serif;}#webpush-custom-prompt-image { width: 64px; position: absolute; top: 15px; left: 15px;}#webpush-custom-prompt-title { margin-left: 90px; margin-top: 4px; font-size: 13px; font-weight: bold; line-height: 18px;}#webpush-custom-prompt-text { font-size: 10px; margin-left: 90px; margin-top: 5px; line-h

In [50]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options

def get_page_rate(url):
    options = Options()
    options.headless = True

    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=options)

    driver.get(url)
    driver.implicitly_wait(5)
    html = driver.page_source
    driver.quit()
    soup = BeautifulSoup(html, "html.parser")

    return soup

def get_rate(url):

    soup = get_page_rate(url)

    # Récupération des notes
    rate = []
    code = soup.find_all("span", class_="player-stat-value")
    for c in code:
        if c.text != '':
            rate.append(c.text)
        else:
            rate.append("NaN")
    nb = len(rate)

    # Récupération des nom des joueurs
    player = []
    code = soup.find_all("span", class_="player-name")
    for c in code:
        if c.text != '':
            player.append(c.text)
    player = player[:nb]
    
    # Récupération du nom du match
    match = re.search(r'202\d-202\d-(.+)', url)
    vec_match = [match.group(1)]*nb

    # Création du dataframe
    df_rate = pd.DataFrame({'Match': vec_match, 'Player': player, 'Rate': rate})

    return df_rate

In [52]:
url = 'https://fr.whoscored.com/matches/1824847/live/france-ligue-1-2024-2025-auxerre-strasbourg'
get_rate(url)

,Match,Player,Rate
0,auxerre-strasbourg,Oppegård,6.3
1,auxerre-strasbourg,Sinayoko,5.9
2,auxerre-strasbourg,Onaiwu,6.0
3,auxerre-strasbourg,Joly,6.0
4,auxerre-strasbourg,Massengo,6.2
5,auxerre-strasbourg,Ayé,NaN
6,auxerre-strasbourg,Percin,NaN
7,auxerre-strasbourg,Dioussé,NaN
8,auxerre-strasbourg,Diomandé,NaN
9,auxerre-strasbourg,Léon,6.3
